# What this script does


---



```
input > weight > hidden layer 1 (activation function) > weights > hidden layer 2 (activation function) > weights > output layer

compare output ot intended output > cost function (cross entropy) optimization function (optimizer) > minimize cost ( AdamOptimizer....SGD, AdaGrad)

backpropagation

feed foward + backprop = epoch
```





In [0]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

# Classes in Mnist Data


---

## Ten classes, 0-9


```
0 = 0
1 = 1
2 = 2
3 = 3
```
## Actually it equates to:


```
0 = [1,0,0,0,0,0,0,0,0,0] # 1's repsent if the pixel is hot or in other words on.
1 = [0,1,0,0,0,0,0,0,0,0]
2 = [0,0,1,0,0,0,0,0,0,0]
3 = [1,0,0,1,0,0,0,0,0,0]

```





In [41]:
mnist = input_data.read_data_sets('/tmp/data/', one_hot=True)

n_nodes_hl1 = 500
n_nodes_hl2 = 500
n_nodes_hl3 = 500

n_classes = 10
batch_size = 100

# matrix is height by width
# height x width
# flat/squash to 784 values; turning into string of values
# x is data, y is label
x = tf.placeholder('float', [None, 784])
y = tf.placeholder('float')

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


# The Model


---
Building the computation graph of tensor flow model and the neural network model

In [0]:
def neural_network_model(data):
  
    # create the variables for the layers
    
    hidden_1_layer = {'weights':tf.Variable(tf.random_normal([784, n_nodes_hl1])),
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl1]))}

    hidden_2_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl1, n_nodes_hl2])),
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl2]))}

    hidden_3_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl2, n_nodes_hl3])),
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl3]))}

    output_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl3, n_classes])),
                    'biases':tf.Variable(tf.random_normal([n_classes]))}
    
    # (input data * weights) + biases - model for each layer
    
    l1 = tf.add(tf.matmul(data,hidden_1_layer['weights']), hidden_1_layer['biases'])
    l1 = tf.nn.relu(l1) # .relu is activation function

    l2 = tf.add(tf.matmul(l1,hidden_2_layer['weights']), hidden_2_layer['biases'])
    l2 = tf.nn.relu(l2) # .relu is activation function

    l3 = tf.add(tf.matmul(l2,hidden_3_layer['weights']), hidden_3_layer['biases'])
    l3 = tf.nn.relu(l3) # .relu is activation function

    output = tf.matmul(l3,output_layer['weights']) + output_layer['biases']

    return output

# Training the Model

---

Specify what to run through the model

In [43]:
def train_neural_network(x):
  prediction = neural_network_model(x)
  cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y) )
  optimizer = tf.train.AdamOptimizer().minimize(cost) # minimizes cost
  
  # cycles of feed foward + backprop (fixing weights)
  hm_epochs = 10
  
  with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    
    # train network
    for epoch in range(hm_epochs):
      epoch_loss = 0
      for _ in range(int(mnist.train.num_examples/batch_size)):
        epoch_x, epoch_y = mnist.train.next_batch(batch_size)
        _, c = sess.run([optimizer, cost], feed_dict = {x: epoch_x, y: epoch_y})
        epoch_loss += c
      print('Epoch', epoch, 'completed out of', hm_epochs, 'loss:', epoch_loss)
    
    # once optimized weights
    # run them through model
    correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
    
    accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
    
    print('Accuracy:', accuracy.eval({x: mnist.test.images, y: mnist.test.labels}))
  

train_neural_network(x)

Epoch 0 completed out of 10 loss: 1935058.148284912
Epoch 1 completed out of 10 loss: 402867.47800827026
Epoch 2 completed out of 10 loss: 213101.25807976723
Epoch 3 completed out of 10 loss: 128181.90613675117
Epoch 4 completed out of 10 loss: 77189.83758546412
Epoch 5 completed out of 10 loss: 48890.68161687968
Epoch 6 completed out of 10 loss: 32051.227833879442
Epoch 7 completed out of 10 loss: 25532.522573613845
Epoch 8 completed out of 10 loss: 20699.310237876176
Epoch 9 completed out of 10 loss: 15710.198667769173
Accuracy: 0.9497
